## Federal Courts Project Guide

The ultimate goal of this project is to build a centralized database of federal judgeships across the 13 district appellate courts and the 96 District courts in the United States. Because of the wealth of data involved, and the fact that much of this data is Scattered across many pages and sites, the first step involves researching the domain, and developing a focus and range of data you want to obtain and make available.

Here are three possible angles:

1. Current judgeships, vacancies, and nomination proceedings: with this focus you would download tables the Recent vacancies and appointments, and go further into nomination procedures and Q&A's. This would entail a combination of scraping, conversions of PDFs, in using regular expressions to parse the PDFs.

2. Historical judgeships: with this focus you examined changes in federal judgeships over a certain period of time (perhaps 10 to 20 years). This would entail mainly the scraping of many pages and the integration of data about specific judges, ordered by district.

3. Recent Nominations and confirmations:  this to focus specifically on judges newly nominated or appointed under the current administration. The focus would be more directly on the nomination hearings (Q&As), as well as the search for other data sources regarding the judges--news articles, opinions, writings by the judges.



Your primary goal by Tuesday is to come up with a specific research question: what kind of knowledge do you want to investigate, build and make available through this project. What are the central units of analysis? What do you want to reveal about the federal courts?

Your secondary goal is to view the primary source pages and begin scraping. You do not have to have your central research question right at the beginning of the scraping, but it may help to have a direction.

You're goal by Thursday is to have a finalize architecture for your dataframe(s), any finalized list of sources that you Will scrape/obtain.

**Data Architecture**
The question of architecture is central to this project. Because of the many possible angles, and the highly decentralized state of the primary source data, there is a wide range of designs for tables, rows, columns. You may want to begin scraping some of the main pages to get more familiar with what kind of rows and columns might be involved.

**Interpretive architecture**
This depends I how focused your data frame will be. If you pick specific districts, judges and/or confirmation hearings you may want to do more human reading to assess different ways the framing the politics/legal perspective of the judge or the district's decisions. If you choose to cast a wider net data, then you will want to focus on more quantitative categories for framing this: judges age, District, background, length of appointment, length of vacancy, Number of vacancies, etc.



### Coding considerations:
While there is a great amount of data available, much of it is distributed across multiple pages, sometimes and inconsistent format. If you're interested in scraping nominations and downloading PDFs, you will need to at least briefly (or entirely) use **selenium**. If you want to use beautiful soup, you will have to download links, and the loop through multiple pages to get a complete data set--unless your focus is more specific.

### Reasons to choose this project

Reportedly, Trump administration faces difficulties at the Senate confirmation. More than 300 key positions need to be confirmed by the Senate.

https://www.washingtonpost.com/graphics/politics/trump-administration-appointee-tracker/database/

For example, at the U.S. Department of State, many senior official position are vacant.

https://www.state.gov/misc/19232.htm

More severely, the Federal Reserve System has only 3 board members (4 are vacant)!

https://www.federalreserve.gov/aboutthefed/bios/board/default.htm

By investigating the federal judicial nomination process, I'd like to find answers to the following questions:

* How serious are judicial vacancies compared with previous presidencies?
* Do high vacancies cause any problem?
* Why does Trump administration's nomination process go wrong overall?


### STEP 1
Scrape the first page of judicial vacancies:

http://www.uscourts.gov/judges-judgeships/judicial-vacancies/current-judicial-vacancies

#### Import your scraping libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

#### Write your scraping code here

In [2]:
response = requests.get('http://www.uscourts.gov/judges-judgeships/judicial-vacancies/current-judicial-vacancies')
doc = BeautifulSoup(response.text, 'html.parser')

vacancies = doc.find_all('tr')[1:]

list_of_vacancies = []

for vacancy in vacancies:
    vacancy_dic = {}
    court = vacancy.find_all('td')[0].text
    incumbent = vacancy.find_all('td')[1].text
    vacancy_reason = vacancy.find_all('td')[2].text
    vacancy_date = vacancy.find_all('td')[3].text
    nominee = vacancy.find_all('td')[4].text
    nomination_date = vacancy.find_all('td')[5].text
    
    vacancy_dic['Court'] = court
    vacancy_dic['Incumbent'] = incumbent
    vacancy_dic['Vacancy Reason'] = vacancy_reason
    vacancy_dic['Vacancy Date'] = vacancy_date
    vacancy_dic['Nominee'] = nominee
    vacancy_dic['Nomination Date'] = nomination_date
    list_of_vacancies.append(vacancy_dic)
    
df_vacancy = pd.DataFrame(list_of_vacancies)
df_vacancy.head()

,Court,Incumbent,Nomination Date,Nominee,Vacancy Date,Vacancy Reason
0,01 - MA,"O'Toole Jr.,George A.",,,01/01/2018,Senior
1,01 - MA,"Woodlock,Douglas P.",,,06/01/2015,Senior
2,01 - ME,"Woodcock Jr.,John A.",04/10/2018,"Walker,Lance E.",06/27/2017,Senior
3,01 - PR,"Fuste,Jose Antonio",04/12/2018,"Arias-Marxuach,Raul M.",06/01/2016,Retired
4,01 - RI,"Lisi,Mary M.",04/12/2018,"McElroy,Mary S.",10/01/2015,Senior


In [3]:
df_vacancy['Circuit'] = df_vacancy.Court.str.extract('(\d\d)')
df_vacancy['District'] = df_vacancy.Court.str.extract('([A-Z][A-Z-]+)')
df_vacancy.head()

,Court,Incumbent,Nomination Date,Nominee,Vacancy Date,Vacancy Reason,Circuit,District
0,01 - MA,"O'Toole Jr.,George A.",,,01/01/2018,Senior,01,MA
1,01 - MA,"Woodlock,Douglas P.",,,06/01/2015,Senior,01,MA
2,01 - ME,"Woodcock Jr.,John A.",04/10/2018,"Walker,Lance E.",06/27/2017,Senior,01,ME
3,01 - PR,"Fuste,Jose Antonio",04/12/2018,"Arias-Marxuach,Raul M.",06/01/2016,Retired,01,PR
4,01 - RI,"Lisi,Mary M.",04/12/2018,"McElroy,Mary S.",10/01/2015,Senior,01,RI


In [4]:
df_vacancy.Circuit.value_counts(dropna=False)

09     27
05     21
11     19
03     18
02     13
NaN    10
04      8
07      8
06      8
10      7
08      6
01      5
Name: Circuit, dtype: int64

### STEP 2
Scrape the first page of judicial confirmations:

http://www.uscourts.gov/judges-judgeships/judicial-vacancies/confirmation-listing


In [4]:
response = requests.get('http://www.uscourts.gov/judges-judgeships/judicial-vacancies/confirmation-listing')
doc = BeautifulSoup(response.text, 'html.parser')

confirmations = doc.find_all('tr')[1:]

list_of_confirmations = []

for confirmation in confirmations:
    confirmation_dic = {}
    
    nominee = confirmation.find_all('td')[0].text
    nomination_date = confirmation.find_all('td')[1].text
    confirmation_date = confirmation.find_all('td')[2].text
    court = confirmation.find_all('td')[3].text
    incumbent = confirmation.find_all('td')[4].text
    vacancy_reason = confirmation.find_all('td')[5].text
    vacancy_date = confirmation.find_all('td')[6].text
    
    confirmation_dic['Nominee'] = nominee
    confirmation_dic['Nomination Date'] = nomination_date
    confirmation_dic['Confirmation Date'] = confirmation_date
    confirmation_dic['Court'] = court
    confirmation_dic['Incumbent'] = incumbent
    confirmation_dic['Vacancy Reason'] = vacancy_reason
    confirmation_dic['Vacancy Date'] = vacancy_date
    list_of_confirmations.append(confirmation_dic)
    
df_confirmation = pd.DataFrame(list_of_confirmations)
df_confirmation.head()

,Confirmation Date,Court,Incumbent,Nomination Date,Nominee,Vacancy Date,Vacancy Reason
0,11/02/2017,03 - CCA,"Rendell,Marjorie O.",06/19/2017,"Bibas,Stephanos",07/01/2015,Senior
1,11/16/2017,04 - SC,"Anderson Jr.,Joseph F.",08/03/2017,"Coggins Jr.,Donald C.",11/16/2014,Senior
2,03/01/2018,04 - SC,"Currie,Cameron McGowan",08/03/2017,"Quattlebaum Jr.,A. Marvin",10/03/2013,Senior
3,04/24/2018,05 - CCA,"Davis,W. Eugene",01/08/2018,"Duncan,Stuart Kyle",12/31/2016,Senior
4,05/09/2018,05 - CCA,"Clement,Edith Brown",01/08/2018,"Engelhardt,Kurt D.",05/09/2018,Senior


In [5]:
df_confirmation['Circuit'] = df_confirmation.Court.str.extract('(\d\d)')
df_confirmation['District'] = df_confirmation.Court.str.extract(r'([A-Z]+)')
df_confirmation.head()

,Confirmation Date,Court,Incumbent,Nomination Date,Nominee,Vacancy Date,Vacancy Reason,Circuit,District
0,11/02/2017,03 - CCA,"Rendell,Marjorie O.",06/19/2017,"Bibas,Stephanos",07/01/2015,Senior,03,CCA
1,11/16/2017,04 - SC,"Anderson Jr.,Joseph F.",08/03/2017,"Coggins Jr.,Donald C.",11/16/2014,Senior,04,SC
2,03/01/2018,04 - SC,"Currie,Cameron McGowan",08/03/2017,"Quattlebaum Jr.,A. Marvin",10/03/2013,Senior,04,SC
3,04/24/2018,05 - CCA,"Davis,W. Eugene",01/08/2018,"Duncan,Stuart Kyle",12/31/2016,Senior,05,CCA
4,05/09/2018,05 - CCA,"Clement,Edith Brown",01/08/2018,"Engelhardt,Kurt D.",05/09/2018,Senior,05,CCA


In [6]:
df_confirmation.Circuit.value_counts(dropna=False)

06     9
05     8
11     5
NaN    5
07     4
10     4
08     3
04     2
03     1
09     1
Name: Circuit, dtype: int64

### STEP 3
Investigate the judicial committee's confirmation postings:

https://www.judiciary.senate.gov/nominations/confirmed

This is relatively straightforward, except that the most interesting information is possibly PDFs of the questionnaires for each candidate. To get the PDFs you need to use selenium (see step 4), but first look this data and assess whether you think it will be useful to you. If so, I will give you the code you need to obtain the PDFs and convert them to text. You can then parse them using regular expressions.

Don't necessarily code here

Think about where you're going first

And read below

### STEP 4
Investigate the judicial committee's hearings on nominees: 

https://www.judiciary.senate.gov/hearings

This one is pretty tricky. It is where you can find PDFs with Q&A's from confirmation hearings. It is a multiple page scrape just to get links to various nomination pages, which then have links to PDFs, which is then have redirects to download the PDFs (you have to use selenium here). 

But before you do the scrape just go through the hearings pages by hand and click on where it says "Nominations". Look at the different Q&A's available and see if you think they will be useful to you. If they will be I can give you most of the code you will need to get the PDFs. Also, I have uploaded a file on slack of one hearings PDFs along with text conversions of them. Take a look at the text conversions, because you'll need to parse them using regular expressions.

If you are interested in more historical data, look into the information on these links:

Archives of vacancies/confirmations (if you want to build more historical data)

http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies

Present and past judges including resumes:

Appeals courts:
https://www.fjc.gov/history/courts/u.s.-court-appeals-district-columbia-circuit-justices-and-judges

District courts:
https://www.fjc.gov/history/courts/u.s.-district-courts-and-federal-judiciary

#### Think about your focus and what your ultimate architecture should be
#### More to come...

** I'm interested in historical aspect of vacancies and start scraping archives. **

In [7]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

In [8]:
driver = webdriver.Chrome()
driver.get('http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies')
archives = driver.find_elements_by_tag_name('tbody')[1].find_elements_by_tag_name('a')
year_urls = []
for archive in archives:
    year_dic = {}
    year = archive.text
    year_url = archive.get_attribute('href')
    year_dic['Year'] = year
    year_dic['Year URL'] = year_url
    year_urls.append(year_dic)
pd.set_option('display.max_colwidth', -1)
df_year = pd.DataFrame(year_urls)

In [9]:
df_year

,Year,Year URL
0,2018,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2018
1,2017,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2017
2,2016,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2016
3,2015,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2015
4,2014,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2014
5,2013,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2013
6,2012,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2012
7,2011,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2011
8,2010,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2010
9,2009,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2009


In [10]:
driver = webdriver.Chrome()
month_urls = []
for url in df_year['Year URL'][:20]:
    driver.get(url)
    year = driver.find_element_by_tag_name('caption').text.split(' ')[1]
    month_archives = driver.find_element_by_tag_name('tbody').find_elements_by_tag_name('tr')
    for month_archive in month_archives:
        month_dic = {}
        month = month_archive.find_element_by_tag_name('td').text
        vacancy_summary = month_archive.find_elements_by_tag_name('a')[0].get_attribute('href')
        judicial_vacancy_list = month_archive.find_elements_by_tag_name('a')[1].get_attribute('href')
        month_dic['Year'] = year
        month_dic['Month'] = month
        month_dic['Vacancy Summary'] = vacancy_summary
        month_dic['Judicial Vacancy List'] = judicial_vacancy_list
        try:
            judicial_emergencies = month_archive.find_elements_by_tag_name('a')[2].get_attribute('href')
            future_judicial_vacancies = month_archive.find_elements_by_tag_name('a')[3].get_attribute('href')
            judicial_confirmations = month_archive.find_elements_by_tag_name('a')[4].get_attribute('href')
            month_dic['Judicial Emergencies'] = judicial_emergencies
            month_dic['Future Judicial Vacancies'] = future_judicial_vacancies
            month_dic['Judicial Confirmations'] = judicial_confirmations
        except:
            pass
        month_urls.append(month_dic)

month_urls
df_month = pd.DataFrame(month_urls)

In [11]:
df_month

,Future Judicial Vacancies,Judicial Confirmations,Judicial Emergencies,Judicial Vacancy List,Month,Vacancy Summary,Year
0,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2018/07/future,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2018/07/confirmations,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2018/07/emergencies,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2018/07/vacancies,July,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2018/07/summary,2018
1,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2018/06/future,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2018/06/confirmations,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2018/06/emergencies,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2018/06/vacancies,June,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2018/06/summary,2018
2,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2018/05/future,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2018/05/confirmations,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2018/05/emergencies,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2018/05/vacancies,May,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2018/05/summary,2018
3,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2018/04/future,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2018/04/confirmations,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2018/04/emergencies,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2018/04/vacancies,April,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2018/04/summary,2018
4,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2018/03/future,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2018/03/confirmations,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2018/03/emergencies,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2018/03/vacancies,March,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2018/03/summary,2018
5,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2018/02/future,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2018/02/confirmations,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2018/02/emergencies,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2018/02/vacancies,February,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2018/02/summary,2018
6,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2018/01/future,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2018/01/confirmations,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2018/01/emergencies,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2018/01/vacancies,January,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-judicial-vacancies/2018/01/summary,2018
7,http://www.uscourts.gov/judges-judgeships/judicial-vacancies/archive-

** I got all urls on the web site and tried to scrape every page of judicial vacancies as below. There are some interesting observations. Under the Trump administration, overall vacancies have increased by nealy 40. On the other hand, vacancies at the court of appeal have decreased a bit. **

** But I was stuck here because link urls are mix of HTML and PDF, page layouts differ between presidencies... **

** So I change my strategy to use another database.**

In [13]:
driver = webdriver.Chrome()
list_of_summaries = []

for url in df_month['Vacancy Summary'][:18]:
    driver.get(url)
    try:
        date = driver.find_element_by_class_name('date-display-single').text
        summary = driver.find_elements_by_tag_name('tr')[1:7]

        for item in summary:
            summary_dic = {}
            court = item.find_elements_by_tag_name('td')[0].text
            authorized_judgeship = item.find_elements_by_tag_name('td')[1].text
            vacancies = item.find_elements_by_tag_name('td')[2].text
            nominees_pending = item.find_elements_by_tag_name('td')[3].text
            nominees_pending_for_future_vacancies = item.find_elements_by_tag_name('td')[4].text
            
            summary_dic['Date'] = date
            summary_dic['Court'] = court
            summary_dic['Authorized Judgeships'] = authorized_judgeship
            summary_dic['Vacancies'] = vacancies
            summary_dic['Nominees Pending'] = nominees_pending
            summary_dic['Nominees Pending for Future Vacancies'] = nominees_pending_for_future_vacancies
            list_of_summaries.append(summary_dic)
            
    except:
        pass

df_summary = pd.DataFrame(list_of_summaries)

In [14]:
df_summary.head(18)

,Authorized Judgeships,Court,Date,Nominees Pending,Nominees Pending for Future Vacancies,Vacancies
0,179,US Court of Appeals,07/01/2018,8,5,15
1,677,US District Courts (includes territorial courts*),07/01/2018,76,0,129
2,9,US Court of International Trade,07/01/2018,2,0,2
3,16,US Court of Federal Claims*,07/01/2018,3,0,7
4,9,US Supreme Court,07/01/2018,0,0,0
5,890,Total,07/01/2018,89,5,153
6,179,US Court of Appeals,06/01/2018,7,3,14
7,677,US District Courts (includes territorial courts*),06/01/2018,70,2,126
8,9,US Court of International Trade,06/01/2018,0,0,2
9,16,US Court of Federal Claims*,06/01/2018,3,0,6


In [15]:
df_summary[df_summary.Court == 'Total']

,Authorized Judgeships,Court,Date,Nominees Pending,Nominees Pending for Future Vacancies,Vacancies
5,890,Total,07/01/2018,89,5,153
11,890,Total,06/01/2018,80,5,148
17,890,Total,05/01/2018,68,4,148
23,890,Total,04/01/2018,54,3,149
29,890,Total,03/01/2018,58,3,148
35,890,Total,02/01/2018,50,3,145
41,890,Total,01/01/2018,51,2,148
47,890,Total,12/01/2017,44,2,144
53,890,Total,11/01/2017,49,3,148
59,890,Total,10/01/2017,47,2,148


In [16]:
df_summary[df_summary.Court == 'US Court of Appeals']

,Authorized Judgeships,Court,Date,Nominees Pending,Nominees Pending for Future Vacancies,Vacancies
0,179,US Court of Appeals,07/01/2018,8,5,15
6,179,US Court of Appeals,06/01/2018,7,3,14
12,179,US Court of Appeals,05/01/2018,9,3,18
18,179,US Court of Appeals,04/01/2018,7,3,17
24,179,US Court of Appeals,03/01/2018,7,3,16
30,179,US Court of Appeals,02/01/2018,5,2,17
36,179,US Court of Appeals,01/01/2018,5,2,16
42,179,US Court of Appeals,12/01/2017,7,2,17
48,179,US Court of Appeals,11/01/2017,11,3,21
54,179,US Court of Appeals,10/01/2017,9,2,21
